In [ ]:
# ref: https://www.analyticsvidhya.com/blog/2020/01/first-text-classification-in-pytorch/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/Case Presentation 1

/content/drive/My Drive/Case Presentation 1


In [ ]:
import torch
#!pip install torchtext
from torchtext import data

In [ ]:
torch.manual_seed(2020);
torch.backends.cudnn.deterministic = True

In [ ]:
#!pip install spacy
# !python -m spacy download en
import spacy
TEXT = data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.float,batch_first=True)

In [ ]:
fields = [(None, None), ('text',TEXT),('label', LABEL)]

In [ ]:
training_data=data.TabularDataset(path = 'data.csv',format = 'csv',fields = fields,skip_header = True)

print(vars(training_data.examples[17]))

NameError: ignored

In [ ]:
import random
train_data, valid_data = training_data.split(split_ratio=0.7, random_state = random.seed(2023))

In [ ]:
#initialize glove embeddings
TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
print(TEXT.vocab.stoi) 

Size of TEXT vocabulary: 746
Size of LABEL vocabulary: 4
[(',', 576), (':', 564), ('and', 235), ('the', 182), ('_', 179), ('*', 178), ('of', 171), ('was', 171), ('-', 135), ('to', 132)]
defaultdict(<function _default_unk_index at 0x7f7592e09730>, {'<unk>': 0, '<pad>': 1, ',': 2, ':': 3, 'and': 4, 'the': 5, '_': 6, '*': 7, 'of': 8, 'was': 9, '-': 10, 'to': 11, 'with': 12, 'a': 13, 'in': 14, '/': 15, 'patient': 16, '.': 17, 'for': 18, ')': 19, '(': 20, 'on': 21, '.She': 22, 'is': 23, 'no': 24, '.The': 25, 'at': 26, 'pain': 27, 'PO': 28, 'her': 29, ';': 30, 'HISTORY': 31, 'or': 32, 'she': 33, 'day': 34, 'had': 35, 'left': 36, 'DISCHARGE': 37, 'DIAGNOSIS': 38, 'Status': 39, 'an': 40, 'by': 41, 'has': 42, 'Date': 43, 'NORMAL': 44, 'history': 45, "'s": 46, '12:00:00': 47, 'MG': 48, 'mg': 49, 'showed': 50, 'Dr.': 51, '2': 52, 'OF': 53, 'The': 54, 'were': 55, 'DATE': 56, 'M.D.': 57, '.He': 58, 'chest': 59, 'normal': 60, '1': 61, 'PROCEDURES': 62, '.PAST': 63, 'daily': 64, 'right': 65, '.Her': 

In [ ]:
#check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#set batch size
BATCH_SIZE = 7

#Load an iterator
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    device = device)
device

device(type='cuda')

In [ ]:
import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

        #activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        self.embedding = self.embedding
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs

In [ ]:
#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 32
num_output_nodes = 4
num_layers = 2
bidirection = True
dropout = 0.2

#instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers, 
                   bidirectional = True, dropout = dropout)

In [ ]:
#architecture
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

#Initialize the pretrained embedding
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)

classifier(
  (embedding): Embedding(746, 100)
  (lstm): LSTM(100, 32, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=64, out_features=4, bias=True)
  (act): Sigmoid()
)
The model has 134,252 trainable parameters
torch.Size([746, 100])


In [ ]:
import torch.optim as optim

#define optimizer and loss
optimizer = optim.Adam(model.parameters())
#optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
#criterion = nn.BCELoss()
criterion = nn.CrossEntropyLoss()

#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    #rounded_preds = torch.round(preds)
    
    #correct = (rounded_preds == y).float()
    correct = (preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
#push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        
        #retrieve text and no. of words
        text, text_lengths = batch.text   
        text_lengths = text_lengths.cuda()
            
        #convert to 1D tensor
        #predictions = model(text, text_lengths).squeeze()
        
        predictions = model(text, text_lengths).squeeze() 

        #predictions = model(text, text_lengths)
        #predictions = torch.max(predictions, 1)[1]
        #predictions = predictions.type(torch.FloatTensor)
        #predictions = predictions.cuda()
        #predictions = predictions.requires_grad_()

        #compute the loss
        loss = criterion(predictions, batch.label.type(torch.LongTensor).cuda())
        
        #compute the binary accuracy
        acc = binary_accuracy(torch.max(predictions, 1)[1].type(torch.FloatTensor), batch.label.cpu())   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            #retrieve text and no. of words
            text, text_lengths = batch.text
            text = text.cuda()
            text_lengths = text_lengths.cuda()

            #convert to 1d tensor
            #predictions = model(text, text_lengths).squeeze()

            predictions = model(text, text_lengths).squeeze()  

            #predictions = model(text, text_lengths)
            #predictions = torch.max(predictions, 1)[1]
            #predictions = predictions.type(torch.FloatTensor) 
            #predictions = predictions.cuda()
            #predictions = predictions.requires_grad_()

            #compute loss and accuracy
            loss = criterion(predictions, batch.label.type(torch.LongTensor).cuda())
            acc = binary_accuracy(torch.max(predictions, 1)[1].type(torch.FloatTensor), batch.label.cpu())
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 500
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    print(str(epoch) + f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

0	Train Loss: 1.388 | Train Acc: 21.43%
	 Val. Loss: 1.383 |  Val. Acc: 17.14%
1	Train Loss: 1.386 | Train Acc: 28.57%
	 Val. Loss: 1.383 |  Val. Acc: 17.14%
2	Train Loss: 1.384 | Train Acc: 28.57%
	 Val. Loss: 1.385 |  Val. Acc: 17.14%
3	Train Loss: 1.381 | Train Acc: 28.57%
	 Val. Loss: 1.385 |  Val. Acc: 17.14%
4	Train Loss: 1.379 | Train Acc: 28.57%
	 Val. Loss: 1.386 |  Val. Acc: 17.14%
5	Train Loss: 1.375 | Train Acc: 28.57%
	 Val. Loss: 1.387 |  Val. Acc: 17.14%
6	Train Loss: 1.371 | Train Acc: 28.57%
	 Val. Loss: 1.387 |  Val. Acc: 17.14%
7	Train Loss: 1.365 | Train Acc: 28.57%
	 Val. Loss: 1.386 |  Val. Acc: 17.14%
8	Train Loss: 1.358 | Train Acc: 28.57%
	 Val. Loss: 1.386 |  Val. Acc: 17.14%
9	Train Loss: 1.351 | Train Acc: 28.57%
	 Val. Loss: 1.387 |  Val. Acc: 17.14%
10	Train Loss: 1.345 | Train Acc: 25.00%
	 Val. Loss: 1.386 |  Val. Acc: 17.14%
11	Train Loss: 1.331 | Train Acc: 32.14%
	 Val. Loss: 1.379 |  Val. Acc: 24.29%
12	Train Loss: 1.320 | Train Acc: 35.71%
	 Val. Lo

KeyboardInterrupt: ignored

In [ ]:
# #load weights
# path='/content/saved_weights.pt'
# model.load_state_dict(torch.load(path));
# model.eval();

# #inference 
# import spacy
# nlp = spacy.load('en')

# def predict(model, sentence):
#     tokenized = [tok.text for tok in nlp.tokenizer(sentence)]  #tokenize the sentence 
#     indexed = [TEXT.vocab.stoi[t] for t in tokenized]          #convert to integer sequence
#     length = [len(indexed)]                                    #compute no. of words
#     tensor = torch.LongTensor(indexed).to(device)              #convert to tensor
#     tensor = tensor.unsqueeze(1).T                             #reshape in form of batch,no. of words
#     length_tensor = torch.LongTensor(length)                   #convert to tensor
#     prediction = model(tensor, length_tensor)                  #prediction 
#     return prediction.item()

In [ ]:
# #make predictions
# predict(model, "Are there any sports that you don't like?")

# #insincere question
# predict(model, "Why Indian girls go crazy about marrying Shri. Rahul Gandhi ji?")